In [1]:
import warnings

import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

# BlazeGraph

In [313]:
blazegraph = pd.read_csv('output/blazegraph.csv')

In [314]:
experiments = set(blazegraph['xp'].values)
for xp in experiments:
    queries = set(blazegraph[(blazegraph['xp'] == xp)]['query'].values)
    timeouts = set(blazegraph[(blazegraph['xp'] == xp) & (blazegraph['status'] == 'timeout')]['query'].values)
    errors = set(blazegraph[(blazegraph['xp'] == xp) & (blazegraph['status'] == 'error')]['query'].values).difference(timeouts)
    oks = queries.difference(timeouts.union(errors))
    blazegraph.loc[(blazegraph['xp'] == xp) & (blazegraph['query'].isin(timeouts)), 'status'] = 'timeout'
    blazegraph.loc[(blazegraph['xp'] == xp) & (blazegraph['query'].isin(timeouts)), 'execution_time'] = 900
    blazegraph.loc[(blazegraph['xp'] == xp) & (blazegraph['query'].isin(timeouts)), 'timeout'] = 1
    blazegraph.loc[(blazegraph['xp'] == xp) & (~blazegraph['query'].isin(timeouts)), 'timeout'] = 0
    blazegraph.loc[(blazegraph['xp'] == xp) & (blazegraph['query'].isin(errors)), 'status'] = 'error'
    blazegraph.loc[(blazegraph['xp'] == xp) & (blazegraph['query'].isin(errors)), 'execution_time'] = 0
    blazegraph.loc[(blazegraph['xp'] == xp) & (blazegraph['query'].isin(errors)), 'error'] = 1
    blazegraph.loc[(blazegraph['xp'] == xp) & (~blazegraph['query'].isin(errors)), 'error'] = 0
blazegraph.loc[:, 'total_execution_time'] = blazegraph['execution_time'] + blazegraph['optimization_time']

In [315]:
blazegraph.loc[~blazegraph['xp'].str.startswith('xp1'), 'xp-root'] = blazegraph['xp'].str.split('-').str[0]

## Global results

In [316]:
blazegraph.groupby(
    by=["endpoint", "xp", "query"]
).mean().groupby(
    by=["endpoint", "xp"]
).sum()[['total_execution_time', 'execution_time', 'optimization_time', 'timeout', 'error']]

total_execution_time  execution_time  optimization_time  \
endpoint   xp                                                               
blazegraph xp1            20338.870527    20338.870527           0.000000   
           xp1-1           4990.062967     4990.062967           0.000000   
           xp1-2          35107.304475    35107.304475           0.000000   
           xp2-0           1463.815566     1373.373355          90.442211   
           xp2-1           1273.633394     1187.880191          85.753203   
           xp2-2           1496.313662     1447.892375          48.421287   
           xp2-3           1421.416730     1375.128117          46.288613   
           xp2-4           1581.679393     1535.404368          46.275025   
           xp3-0           1540.185212     1494.085242          46.099970   
           xp3-1           1788.407774     1738.750193          49.657581   
           xp3-2           1367.101415     1323.975087          43.126328   
           xp3-3           1567.153327     1517.369029          49.784298   
           xp3-4           1885.503577     1843.194756          42.308821   
           xp4-0           1886.386149     1851.435695          34.950454   
           xp4-1           2418.740102     2384.805853          33.934249   
           xp4-2           1984.802578     1923.424402          61.378177   
           xp4-3           1892.119913     1857.165867          34.954046   
           xp4-4           2050.323327     2017.301589          33.021737   
           xp5-0           1553.026644     1217.507937         335.518707   
           xp5-1           1682.029648     1349.517200         332.512448   
           xp5-2           1544.527324     1232.702372         311.824952   
           xp5-3           1619.286780     1310.521533         308.765247   
           xp5-4           1327.711400     1020.641377         307.070023   
           xp6-0           1733.049753     1436.963541         296.086212   
           xp6-1           1762.502040     1470.189552         292.312488   
           xp6-2           1690.586501     1405.309273         285.277228   
           xp6-3           1433.188216     1129.688636         303.499580   
           xp6-4           1297.775791     1012.114885         285.660907   
           xp7-0           2082.609861     1841.495269         241.114592   
           xp7-1           2098.599368     1861.882010         236.717357   
           xp7-2           2081.546756     1841.375715         240.171041   
           xp7-3           2070.426718     1834.937409         235.489309   
           xp7-4           2106.701940     1856.100802         250.601138   
           xp8-0           1876.175896     1818.452113          57.723783   
           xp8-1           1415.255719     1355.607415          59.648304   
           xp8-2           1513.642480     1456.248879          57.393600   
           xp8-3           1799.112970     1740.792502          58.320469   
           xp8-4           1573.694727     1518.732707          54.962020   
           xp9-0           1909.278726     1546.517246         362.761479   
           xp9-1           2359.483645     2001.195661         358.287983   
           xp9-2           1793.990010     1433.797908         360.192103   
           xp9-3           1794.564308     1438.545720         356.018589   
           xp9-4           1593.960472     1227.705979         366.254493   

                  timeout  error  
endpoint   xp                     
blazegraph xp1       15.0    9.0  
           xp1-1      0.0    0.0  
           xp1-2     36.0   64.0  
           xp2-0      0.0    0.0  
           xp2-1      0.0    0.0  
           xp2-2      0.0    0.0  
           xp2-3      0.0    0.0  
           xp2-4      0.0    0.0  
           xp3-0      0.0    0.0  
           xp3-1      0.0    0.0  
           xp3-2      0.0    0.0  
           xp3-3      0.0    0.0  
           xp3-4      0.0    0.0  
           xp4-0      0.0  

In [317]:
blazegraph[~blazegraph['xp'].str.startswith('xp1')].groupby(
    by=["endpoint", "xp-root", "xp", "query"]
).mean().groupby(
    by=["endpoint", "xp-root", "xp"]
).sum().groupby(
    by=["endpoint", "xp-root"]
).mean()[['total_execution_time', 'execution_time', 'optimization_time', 'timeout', 'error']]

total_execution_time  execution_time  optimization_time  \
endpoint   xp-root                                                            
blazegraph xp2               1447.371749     1383.935681          63.436068   
           xp3               1629.670261     1583.474861          46.195400   
           xp4               2046.474414     2006.826681          39.647733   
           xp5               1545.316359     1226.178084         319.138275   
           xp6               1583.420460     1290.853177         292.567283   
           xp7               2087.976928     1847.158241         240.818687   
           xp8               1635.576358     1577.966723          57.609635   
           xp9               1890.255432     1529.552503         360.702929   

                    timeout  error  
endpoint   xp-root                  
blazegraph xp2          0.0    0.0  
           xp3          0.0    0.0  
           xp4          0.2    0.0  
           xp5          0.0    0.0  
           xp6          0.0    0.0  
           xp7          0.0    0.0  
           xp8          0.0    0.0  
           xp9          0.0    0.0

In [318]:
blazegraph[~blazegraph['xp'].str.startswith('xp1')].groupby(
    by=["endpoint", "xp-root", "xp", "query"]
).mean().groupby(
    by=["endpoint", "xp-root", "xp"]
).sum().groupby(
    by=["endpoint", "xp-root"]
).std()[['total_execution_time', 'execution_time', 'optimization_time', 'timeout', 'error']]

total_execution_time  execution_time  optimization_time  \
endpoint   xp-root                                                            
blazegraph xp2                113.537364      128.121147          22.590742   
           xp3                207.066791      206.921163           3.514281   
           xp4                219.005443      221.629554          12.174307   
           xp5                133.766715      127.170585          13.728749   
           xp6                206.219726      206.324390           7.628988   
           xp7                 14.505384       11.309447           5.946144   
           xp8                194.852991      195.008347           1.712529   
           xp9                285.810920      287.792274           3.971569   

                     timeout  error  
endpoint   xp-root                   
blazegraph xp2      0.000000    0.0  
           xp3      0.000000    0.0  
           xp4      0.447214    0.0  
           xp5      0.000000    0.0  
           xp6      0.000000    0.0  
           xp7      0.000000    0.0  
           xp8      0.000000    0.0  
           xp9      0.000000    0.0

## Main plot for BlazeGraph

In [319]:
baseline = blazegraph[blazegraph['xp'] == 'xp1']
baseline.loc[baseline['status'] == 'timeout', 'BlazeGraph (timeout; >900s)'] = 900
baseline.loc[baseline['status'] == 'error', 'BlazeGraph (error)'] = 900
baseline.loc[baseline['status'] == 'ok', 'BlazeGraph'] = baseline['total_execution_time']
baseline = baseline.groupby(by=['query']).mean().reset_index()

experiment = blazegraph[blazegraph['xp'].str.startswith('xp2')]
experiment.loc[:, 'Total Execution Time'] = experiment['total_execution_time']
experiment.loc[:, 'Execution Time'] = experiment['execution_time']
experiment = experiment.groupby(by=['query']).mean().reset_index()

df = baseline.merge(experiment, on=['query'], suffixes=['_baseline', '_xp'])
df.loc[:, 'Wikidata time limit (60s)'] = 60

In [320]:
order = list(df.sort_values(by='total_execution_time_baseline')['query'])
order.reverse()

metrics = [
    'BlazeGraph (timeout; >900s)',
    'BlazeGraph (error)',
    'BlazeGraph',
    'Total Execution Time',
    'Execution Time',
    'Wikidata time limit (60s)'
]
colors = [
    'darkgrey',
    'indianred',
    'lightgray',
    'royalblue',
    'forestgreen',
    'black'
]

base = alt.Chart(df).mark_line().transform_fold(
    metrics, as_=['Measure', 'Value']
).encode(
    alt.X(
        'query:N',
        sort=order,
        axis=alt.Axis(labels=False, ticks=False),
        title='Queries'
    ),
    alt.Color(
        'Measure:N',
        scale=alt.Scale(domain=metrics, range=colors),
        legend = alt.Legend(
            title="Legend",
            labelFontSize=16,
            titleFontSize=18,
            labelLimit=300,
            orient="bottom",
            columns=3
        )
    )
)

baseline_1 = base.mark_bar().transform_filter(
    alt.datum.Measure == metrics[0]
).encode(
    alt.Y(
        'Value:Q', 
        scale=alt.Scale(type='symlog'),
        title='Time [symlog][seconds]'
    )
)

baseline_3 = base.mark_bar().transform_filter(
    alt.datum.Measure == metrics[1]
).encode(
    alt.Y(
        'Value:Q', 
        scale=alt.Scale(type='symlog')
    )
)

baseline_2 = base.mark_bar().transform_filter(
    alt.datum.Measure == metrics[2]
).encode(
    alt.Y(
        'Value:Q', 
        scale=alt.Scale(type='symlog')
    )
)

metric_1 = base.transform_filter(
    alt.datum.Measure == metrics[3]
).encode(
    alt.Y(
        'Value:Q', 
        scale=alt.Scale(type='symlog')
    )
)

metric_2 = base.transform_filter(
    alt.datum.Measure == metrics[4]
).encode(
    alt.Y(
        'Value:Q', 
        scale=alt.Scale(type='symlog')
    )
)

timeout = base.mark_line().transform_filter(
    alt.datum.Measure == metrics[5]
).encode(
    alt.Y(
        'Value:Q', 
        scale=alt.Scale(type='symlog')
    ),
    strokeWidth=alt.value(1)
)

alt.layer(baseline_1, baseline_2, baseline_3, metric_2, metric_1 + timeout).properties(width=800).configure_axis(
    labelFontSize=16,
    titleFontSize=16
)

alt.LayerChart(...)

# Virtuoso

In [305]:
virtuoso = pd.read_csv('output/virtuoso.csv')

In [306]:
experiments = set(virtuoso['xp'].values)
for xp in experiments:
    queries = set(virtuoso[(virtuoso['xp'] == xp)]['query'].values)
    timeouts = set(virtuoso[(virtuoso['xp'] == xp) & (virtuoso['status'] == 'timeout')]['query'].values)
    errors = set(virtuoso[(virtuoso['xp'] == xp) & (virtuoso['status'] == 'error')]['query'].values).difference(timeouts)
    oks = queries.difference(timeouts.union(errors))
    virtuoso.loc[(virtuoso['xp'] == xp) & (virtuoso['query'].isin(timeouts)), 'status'] = 'timeout'
    virtuoso.loc[(virtuoso['xp'] == xp) & (virtuoso['query'].isin(timeouts)), 'execution_time'] = 900
    virtuoso.loc[(virtuoso['xp'] == xp) & (virtuoso['query'].isin(timeouts)), 'timeout'] = 1
    virtuoso.loc[(virtuoso['xp'] == xp) & (~virtuoso['query'].isin(timeouts)), 'timeout'] = 0
    virtuoso.loc[(virtuoso['xp'] == xp) & (virtuoso['query'].isin(errors)), 'status'] = 'error'
    virtuoso.loc[(virtuoso['xp'] == xp) & (virtuoso['query'].isin(errors)), 'error'] = 1
    virtuoso.loc[(virtuoso['xp'] == xp) & (~virtuoso['query'].isin(errors)), 'error'] = 0
virtuoso.loc[:, 'total_execution_time'] = virtuoso['execution_time'] + virtuoso['optimization_time']

In [307]:
virtuoso.loc[~virtuoso['xp'].str.startswith('xp1'), 'xp-root'] = virtuoso['xp'].str.split('-').str[0]

## Global results

In [308]:
virtuoso.groupby(
    by=["endpoint", "xp", "query"]
).mean().groupby(
    by=["endpoint", "xp"]
).sum()[['total_execution_time', 'execution_time', 'optimization_time', 'timeout', 'error']]

total_execution_time  execution_time  optimization_time  \
endpoint xp                                                               
virtuoso xp1              418.776458      414.982931           3.793527   
         xp2-0            354.345836      311.076512          43.269324   
         xp2-1            368.337658      324.821291          43.516367   
         xp2-2            351.887363      308.415145          43.472219   
         xp2-3            379.210780      336.157566          43.053214   
         xp2-4            357.884516      314.843448          43.041068   
         xp3-0            342.020091      303.031453          38.988637   
         xp3-1            356.158176      317.226555          38.931621   
         xp3-2            332.743576      293.801482          38.942094   
         xp3-3            406.176539      367.210214          38.966325   
         xp3-4            343.469077      304.619399          38.849678   
         xp4-0            344.190510      313.275474          30.915036   
         xp4-1            354.555115      323.690352          30.864763   
         xp4-2            343.852406      312.799716          31.052690   
         xp4-3            342.881760      312.178382          30.703378   
         xp4-4            342.945361      312.111851          30.833509   
         xp5-0            572.973937      281.623448         291.350489   
         xp5-1            606.100654      314.866517         291.234137   
         xp5-2            585.665457      293.142233         292.523224   
         xp5-3            605.248585      314.828022         290.420563   
         xp5-4            602.016469      310.475326         291.541143   
         xp6-0            569.383308      301.110659         268.272649   
         xp6-1            584.884723      315.178961         269.705762   
         xp6-2            586.308791      318.444497         267.864294   
         xp6-3            561.979579      293.706873         268.272707   
         xp6-4            548.492893      278.658478         269.834415   
         xp7-0            526.991989      306.613372         220.378618   
         xp7-1            529.264360      309.196474         220.067887   
         xp7-2            530.485194      310.483822         220.001372   
         xp7-3            524.982865      304.667686         220.315180   
         xp7-4            530.670107      310.119210         220.550898   
         xp8-0            370.992946      323.309358          47.683588   
         xp8-1            347.267749      299.577163          47.690586   
         xp8-2            393.716336      345.625817          48.090520   
         xp8-3            352.998315      304.903035          48.095280   
         xp8-4            359.589952      311.128480          48.461471   
         xp9-0            621.263612      303.063914         318.199698   
         xp9-1            636.709584      313.807273         322.902311   
         xp9-2            608.883379      290.395537         318.487842   
         xp9-3            611.360915      306.579022         304.781893   
         xp9-4            590.233812      285.943054         304.290758   

                timeout  error  
endpoint xp                     
virtuoso xp1        0.0    0.0  
         xp2-0      0.0    0.0  
         xp2-1      0.0    0.0  
         xp2-2      0.0    1.0  
         xp2-3      0.0    1.0  
         xp2-4      0.0    0.0  
         xp3-0      0.0    0.0  
         xp3-1      0.0    0.0  
         xp3-2      0.0    1.0  
         xp3-3      0.0    1.0  
         xp3-4      0.0    1.0  
         xp4-0      0.0    1.0  
         xp4-1      0.0    0.0  
         xp4-2      0.0    0.0  
         xp4-3      0.0    0.0  
         xp4-4      0.0    2.0  
         xp5-0      0.0    0.0  
         xp5-1      0.0    0.0  
         xp5-2      0.0    1.0  
         xp5-3      0.0    0.0  
         xp5-4      0.0    1.0  
         xp6-0      0.0    0.0 

In [309]:
virtuoso[~virtuoso['xp'].str.startswith('xp1')].groupby(
    by=["endpoint", "xp-root", "xp", "query"]
).mean().groupby(
    by=["endpoint", "xp-root", "xp"]
).sum().groupby(
    by=["endpoint", "xp-root"]
).mean()[['total_execution_time', 'execution_time', 'optimization_time', 'timeout', 'error']]

total_execution_time  execution_time  optimization_time  \
endpoint xp-root                                                            
virtuoso xp2                362.333231      319.062792          43.270438   
         xp3                356.113492      317.177820          38.935671   
         xp4                345.685030      314.811155          30.873875   
         xp5                594.401020      302.987109         291.413911   
         xp6                570.209859      301.419894         268.789965   
         xp7                528.478903      308.216113         220.262791   
         xp8                364.913060      316.908771          48.004289   
         xp9                613.690260      299.957760         313.732500   

                  timeout  error  
endpoint xp-root                  
virtuoso xp2          0.0    0.4  
         xp3          0.0    0.6  
         xp4          0.0    0.6  
         xp5          0.0    0.4  
         xp6          0.0    0.0  
         xp7          0.0    0.0  
         xp8          0.0    0.8  
         xp9          0.0    0.6

In [310]:
virtuoso[~virtuoso['xp'].str.startswith('xp1')].groupby(
    by=["endpoint", "xp-root", "xp", "query"]
).mean().groupby(
    by=["endpoint", "xp-root", "xp"]
).sum().groupby(
    by=["endpoint", "xp-root"]
).std()[['total_execution_time', 'execution_time', 'optimization_time', 'timeout', 'error']]

total_execution_time  execution_time  optimization_time  \
endpoint xp-root                                                            
virtuoso xp2                 11.331488       11.405563           0.224162   
         xp3                 29.201750       29.187025           0.052909   
         xp4                  4.990857        4.986534           0.126935   
         xp5                 14.560015       14.922538           0.753300   
         xp6                 15.926193       16.254364           0.911263   
         xp7                  2.443190        2.494269           0.226657   
         xp8                 18.359244       18.327372           0.326339   
         xp9                 17.073238       11.544982           8.601082   

                  timeout     error  
endpoint xp-root                     
virtuoso xp2          0.0  0.547723  
         xp3          0.0  0.547723  
         xp4          0.0  0.894427  
         xp5          0.0  0.547723  
         xp6          0.0  0.000000  
         xp7          0.0  0.000000  
         xp8          0.0  0.447214  
         xp9          0.0  0.894427

In [311]:
baseline = virtuoso[(virtuoso['xp'] == 'xp1') & (virtuoso['status'] == 'ok')]
baseline.loc[:, 'Virtuoso'] = baseline['total_execution_time']
baseline = baseline.groupby(by=['query']).mean().reset_index()

experiment = virtuoso[(virtuoso['xp'].str.startswith('xp2')) & (virtuoso['status'] == 'ok')]
experiment.loc[:, 'Total Execution Time'] = experiment['total_execution_time']
experiment.loc[:, 'Execution Time'] = experiment['execution_time']
experiment = experiment.groupby(by=['query']).mean().reset_index()

df = baseline.merge(experiment, on=['query'], suffixes=['_baseline', '_xp'])
df.loc[:, 'Wikidata time limit (60s)'] = 60

In [312]:
order = list(df.sort_values(by='total_execution_time_baseline')['query'])
order.reverse()

metrics = [
    'Virtuoso',
    'Total Execution Time',
    'Execution Time',
    'Wikidata time limit (60s)'
]
colors = [
    'lightgray',
    'royalblue',
    'forestgreen',
    'black'
]

base = alt.Chart(df).mark_line().transform_fold(
    metrics, as_=['Measure', 'Value']
).encode(
    alt.X(
        'query:N',
        sort=order,
        axis=alt.Axis(labels=False, ticks=False),
        title='Queries'
    ),
    alt.Color(
        'Measure:N',
        scale=alt.Scale(domain=metrics, range=colors),
        legend = alt.Legend(
            title="Legend",
            labelFontSize=16,
            titleFontSize=18,
            labelLimit=300,
            orient="bottom"
        )
    )
)

baseline_1 = base.mark_bar().transform_filter(
    alt.datum.Measure == metrics[0]
).encode(
    alt.Y(
        'Value:Q', 
        scale=alt.Scale(type='symlog'),
        title='Time [symlog][seconds]'
    )
)

metric_1 = base.transform_filter(
    alt.datum.Measure == metrics[1]
).encode(
    alt.Y(
        'Value:Q', 
        scale=alt.Scale(type='symlog')
    )
)

metric_2 = base.transform_filter(
    alt.datum.Measure == metrics[2]
).encode(
    alt.Y(
        'Value:Q', 
        scale=alt.Scale(type='symlog')
    )
)

timeout = base.mark_line().transform_filter(
    alt.datum.Measure == metrics[3]
).encode(
    alt.Y(
        'Value:Q', 
        scale=alt.Scale(type='symlog')
    ),
    strokeWidth=alt.value(1)
)

alt.layer(baseline_1, metric_2, metric_1 + timeout).properties(width=800).configure_axis(
    labelFontSize=16,
    titleFontSize=16
)

alt.LayerChart(...)